# Thresholds and Their Impact on Precision and Recall

This notebook is a hands-on lab for exploring how classifiers behave on **imbalanced datasets**. It uses a single 2D synthetic dataset that can be re-generated with different parameters, such as **class imbalance, sample size, and noise**.

In this exercise, you will learn about:

* **Thresholds** and how they affect predicted class labels.
* **Probabilistic classification** — how models output probabilities instead of hard labels.
* **Sample size effects** — how small vs large training sets influence performance metrics.
* **Threshold-independent metrics** — ROC AUC and Average Precision (PR AUC), which summarize model performance without committing to a single threshold.

> Understanding these concepts is essential for choosing the right metric and threshold in real-world, imbalanced classification problems.

---

## 0. Setup / Imports



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    confusion_matrix, classification_report, accuracy_score, precision_score,
    recall_score, f1_score, roc_curve, roc_auc_score, precision_recall_curve,
    average_precision_score
)
from sklearn.calibration import calibration_curve
from sklearn.utils import resample
import warnings
warnings.filterwarnings('ignore')

---

## 1. Helper functions
We’ll create two helpful utilities:
- `generate_data()` to produce the synthetic 2D dataset with controllable parameters.
- `plot_decision_boundary()` to show the model decision regions; it supports passing a
  probability threshold to visualize thresholded decision boundaries for probability-based models.


In [ ]:
def generate_data(
    n_samples=2000,
    weights=(0.95, 0.05),
    class_sep=1.0,
    flip_y=0.01,
    random_state=None
):
    """
    Generate a 2D classification dataset with controllable imbalance and overlap.

    Parameters
    ----------
    n_samples : int
    weights : tuple (neg_weight, pos_weight)
    class_sep : float, larger => easier to separate
    flip_y : float, label noise
    random_state : int or None

    Returns
    -------
    X : ndarray (n_samples, 2)
    y : ndarray (n_samples,)
    """
    X, y = make_classification(
        n_samples=n_samples,
        n_features=2,
        n_redundant=0,
        n_clusters_per_class=1,
        weights=list(weights),
        class_sep=class_sep,
        flip_y=flip_y,
        random_state=random_state
    )
    return X, y


# Decision boundary plotting utility
from matplotlib import cm


def plot_decision_boundary(model, X, y, ax=None, threshold=0.5, proba=True, title=None, mesh_step=0.02):
    """
    Plot a 2D decision boundary for a classifier.

    If model supports `predict_proba` or `decision_function` this function can visualize
    thresholded decision regions. If not (e.g. KNeighbors or decision tree without probabilities),
    set `proba=False` and it will use `model.predict`.

    Parameters
    ----------
    model : fitted classifier
    X, y : data
    ax : matplotlib axis
    threshold : float in [0,1] (used only if proba=True)
    proba : bool - whether to use probabilities/scores
    title : str
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(6,5))
    else:
        fig = ax.figure

    x_min, x_max = X[:, 0].min() - 1.0, X[:, 0].max() + 1.0
    y_min, y_max = X[:, 1].min() - 1.0, X[:, 1].max() + 1.0
    xx, yy = np.meshgrid(np.arange(x_min, x_max, mesh_step), np.arange(y_min, y_max, mesh_step))
    grid = np.c_[xx.ravel(), yy.ravel()]

    try:
        if proba and hasattr(model, 'predict_proba'):
            Z = model.predict_proba(grid)[:, 1]
            Zf = (Z >= threshold).astype(int)
            cmap = cm.RdYlBu
            Zplot = Z.reshape(xx.shape)
            cont = ax.contourf(xx, yy, Zplot, levels=20, alpha=0.4, cmap=cmap)
            ax.contour(xx, yy, Z.reshape(xx.shape), levels=[threshold], colors=['k'], linewidths=1.2)
        elif proba and hasattr(model, 'decision_function'):
            scores = model.decision_function(grid)
            # scale scores to 0..1 using min-max for visualization (not probabilities)
            scores_scaled = (scores - scores.min()) / (scores.max() - scores.min() + 1e-12)
            ax.contourf(xx, yy, scores_scaled.reshape(xx.shape), levels=20, alpha=0.4)
            ax.contour(xx, yy, scores_scaled.reshape(xx.shape), levels=[threshold], colors=['k'], linewidths=1.2)
        else:
            Z = model.predict(grid)
            Zf = Z.reshape(xx.shape)
            ax.contourf(xx, yy, Zf, alpha=0.3, cmap=cm.coolwarm)
    except Exception as e:
        # fallback
        try:
            Z = model.predict(grid)
            ax.contourf(xx, yy, Z.reshape(xx.shape), alpha=0.3, cmap=cm.coolwarm)
        except Exception:
            pass

    # scatter
    scatter = ax.scatter(X[:, 0], X[:, 1], c=y, s=20, edgecolor='k', cmap=cm.coolwarm)
    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_xlabel('x1')
    ax.set_ylabel('x2')
    if title is not None:
        ax.set_title(title)
    return ax



## 2. Generate a base dataset and visualize
We’ll create a default unified dataset: 2000 samples, 95% negative, 5% positive, moderate separation.



In [ ]:
class_sep= 1    # less separation 1 is easy, o → more overlap
flip_y=0.01     #  noisy labels


X, y = generate_data(n_samples=2000, weights=(0.95, 0.05), class_sep= class_sep, flip_y= flip_y, random_state=0)
plt.figure(figsize=(6,5))
plt.scatter(X[:,0], X[:,1], c=y, s=15, cmap='coolwarm', edgecolors='k')
plt.title('Base dataset: 2D synthetic (95/5 imbalance)')
plt.xlabel('x1')
plt.ylabel('x2')
plt.show()


In [ ]:
# show class counts
unique, counts = np.unique(y, return_counts=True)
print(f"Class 0 count: {counts[0]} | Class 1 count: {counts[1]}")


## 3. Train the models
We’ll use **KNN**, **Decision Tree**, **SVM (with probability enabled)**, and **Logistic Regression**.

*Short explanation of Logistic Regression:*  
It is a linear model that predicts the log-odds of the positive class and produces probabilities via the logistic (sigmoid) function.

⚠️ **Note on hyper-parameters:**  
The ones used here are **not** optimal — they’re chosen for clarity. Feel free to experiment by changing them later.




In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=1)

# Standardize for distance-based models (KNN, SVM) — keep pipeline simple here
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

models = {
    'knn': KNeighborsClassifier(n_neighbors=5),
    'tree': DecisionTreeClassifier(max_depth=5, random_state=1),
    'svm': SVC(probability=True, gamma='scale', C=1.0, random_state=1),
    'logreg': LogisticRegression(solver='lbfgs', max_iter=1000, random_state=1)
}

fitted = {}
for name, model in models.items():
    if name in ('knn', 'svm'):
        model.fit(X_train_scaled, y_train)
    else:
        model.fit(X_train, y_train)
    fitted[name] = model

### 🔑 Key Concept: Probabilities and Thresholds

Many classifiers can output **probabilities** (or probability-like scores) for each class, not just hard labels.
By default, most models assign the positive class if the probability ≥ **0.5** (the default threshold).

Thresholds can be adjusted:

* **Lower threshold (e.g., 0.3):** more positives → higher recall, but more false positives.
* **Higher threshold (e.g., 0.7):** fewer positives → higher precision, but more false negatives.

**Why it matters:**
Threshold choice is central to this lab. Metrics like F1, precision, and recall change as the threshold changes. The “best” threshold depends on *context* or *business value*, not just math.

---

### 🔍 First look: probabilities and thresholds

Let’s inspect model outputs. Instead of `predict()`, which returns 0/1, many models provide **predicted probabilities** with `predict_proba()`.

**Example output of the code below:**

```
Sample 0: True=0, Prob=0.00, Pred@0.5=0
```

**How to read it:**

* **Sample 0:** index of the test sample.
* **True=0:** actual class from the test set (`0` = negative).
* **Prob=0.00:** model’s predicted probability of being positive. Close to 0 means very confident it’s negative.
* **Pred@0.5=0:** predicted label using threshold 0.5. Since Prob < 0.5, the model predicts class 0.

**Key takeaways:**

* Probabilities show the model’s confidence, not just a label.
* The predicted label depends on the threshold; adjusting it can flip borderline predictions.
* Understanding this explains why **precision, recall, and F1** vary with threshold.



In [ ]:

# Pick 3 sample points from the test set
X_sample = X_test_scaled[:3]
y_sample = y_test[:3]

for name, model in fitted.items():
    if name in ('knn', 'svm'):
        probs = model.predict_proba(X_sample)[:, 1]
    else:
        probs = model.predict_proba(X_test[:3])[:, 1]

    print(f"\n{name.upper()} predictions (3 sample points):")
    for i, (true_label, prob) in enumerate(zip(y_sample, probs)):
        # simple threshold demonstration
        threshold= 0.5  # between 0 and 1
        pred_default = int(prob >= threshold) # is the probability higher or lower than the threshild
        print(f"Sample {i}: True={true_label}, Prob={prob:.2f}, Pred@{threshold}={pred_default}")


In the code above change the threshold of 0.5 and re-run
```
        pred_default = int(prob >= 0.5)
```






Visualize decision boundaries (default threshold 0.5) for each model. For KNN and tree the `predict_proba` exists for KNN and tree; for tree we used unscaled X; for SVM & KNN we visualized using scaled features but plotted on original X space — this is an approximation (the mesh uses original X coordinates). This is intentionally kept simple for didactic purposes.


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12,10))
axes = axes.ravel()
for ax, (name, model) in zip(axes, fitted.items()):
    if name in ('knn', 'svm'):
        # these were fit on scaled features; create a wrapper to map grid through scaler
        def wrapper_predict_proba(grid):
            gsc = scaler.transform(grid)
            return model.predict_proba(gsc)
        # monkeypatch predict_proba for visualization
        class FakeModel:
            def predict_proba(self, grid):
                return wrapper_predict_proba(grid)
        fake = FakeModel()
        plot_decision_boundary(fake, X, y, ax=ax, title=name.upper())
    else:
        plot_decision_boundary(model, X, y, ax=ax, title=name.upper())
plt.tight_layout()
plt.show()

One more **helper function**: I’ve merged everything into a single function so you can easily explore how different dataset properties impact model behavior.”

In [ ]:
def make_experiment(n_samples=2000, weights=(0.95, 0.05),
                    class_sep=1.0, flip_y=0.01, random_state=0):
    """
    Generate synthetic imbalanced dataset, split train/test,
    scale features where needed, train four classifiers.

    Returns:
    --------
    X_train, X_test, y_train, y_test : arrays
    X_train_scaled, X_test_scaled : arrays (for KNN/SVM)
    fitted : dict of trained models
    """

    # Generate data
    X, y = generate_data(n_samples=n_samples,
                         weights=weights,
                         class_sep=class_sep,
                         flip_y=flip_y,
                         random_state=random_state)

    # Plot
    plt.figure(figsize=(6,5))
    plt.scatter(X[:,0], X[:,1], c=y, s=15,
                cmap='coolwarm', edgecolors='k')
    plt.title(f"Dataset: weights={weights}, sep={class_sep}, noise={flip_y}")
    plt.xlabel('x1'); plt.ylabel('x2')
    plt.show()

    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=1
    )

    # Scale for distance-based models
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Define models
    models = {
        'knn': KNeighborsClassifier(n_neighbors=5),
        'tree': DecisionTreeClassifier(max_depth=5, random_state=1),
        'svm': SVC(probability=True, gamma='scale', C=1.0, random_state=1),
        'logreg': LogisticRegression(solver='lbfgs', max_iter=1000, random_state=1)
    }

    # Fit
    fitted = {}
    for name, model in models.items():
        if name in ('knn', 'svm'):
            model.fit(X_train_scaled, y_train)
        else:
            model.fit(X_train, y_train)
        fitted[name] = model

    return (X_train, X_test, y_train, y_test,
            X_train_scaled, X_test_scaled, fitted)


# 4. Exercises & experiments


### Threshold Effects – Interpretation Exercise

We have provided a code snippet that computes predicted probabilities for each classifier, sweeps thresholds from 0 to 1, calculates metrics (accuracy, precision, recall, F1), and plots them versus threshold.

**Your task:**

1. **Inspect the plots** generated for each classifier.

2. **Answer these exploratory questions:**

   * How do the classifiers **differ in their response** to changes in the threshold?
   * Which classifiers appear **more sensitive** or **less sensitive** to threshold changes?
   * For each metric (accuracy, precision, recall, F1), are there classifiers that consistently perform better or worse? Why might this happen?
   * Are there thresholds where the “ranking” of classifiers changes depending on the metric?

3. **Optional exploration:**

   * Adjust the threshold range (e.g., 0.2–0.8) or the step size and observe whether patterns change.
   * Consider very low or very high thresholds — what happens to false positives vs false negatives?
   * Reflect on how these observations might inform **business or real-world decisions** where the cost of errors differs.

> **Hint:** Focus on understanding **how classifier behavior changes with the threshold** rather than just which one is “best.” Think about patterns, sensitivity, and metric trade-offs.

In [ ]:
from collections import defaultdict



results = {}
for name, model in fitted.items():
    if name in ('knn', 'svm'):
        # use scaled test data
        proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        proba = model.predict_proba(X_test)[:, 1]

    thresholds = np.linspace(0.0, 1.0, 101)
    metrics_vs_t = defaultdict(list)
    for t in thresholds:
        preds = (proba >= t).astype(int)
        metrics_vs_t['threshold'].append(t)
        metrics_vs_t['accuracy'].append(accuracy_score(y_test, preds))
        # guard division by zero in precision/recall
        metrics_vs_t['precision'].append(precision_score(y_test, preds, zero_division=0))
        metrics_vs_t['recall'].append(recall_score(y_test, preds, zero_division=0))
        metrics_vs_t['f1'].append(f1_score(y_test, preds, zero_division=0))

    results[name] = pd.DataFrame(metrics_vs_t)

import matplotlib.cm as cm


metrics = ['accuracy', 'precision', 'recall', 'f1']
fig, axes = plt.subplots(4, 1, figsize=(14, 16))

# Distinct colors and line styles
colors = cm.tab10.colors
line_styles = ['-', '--', '-.', ':']

for i, metric in enumerate(metrics):
    ax = axes[i]
    for j, (name, df) in enumerate(results.items()):
        ax.plot(
            df['threshold'],
            df[metric],
            label=name.upper(),
            color=colors[j % len(colors)],
            linestyle=line_styles[j % len(line_styles)],
            linewidth=2
        )
    ax.set_title(f'{metric.capitalize()} vs Threshold', fontsize=14)  # panel title
    ax.set_ylabel(metric.capitalize())
    ax.set_xlim(0, 1)
    ax.set_xlabel('Threshold')
    ax.grid(True)
    ax.legend(fontsize=10)

plt.tight_layout()
plt.show()



### Question

When you sweep the threshold from 0 to 1, why do **KNN and Decision Tree** maintain non-zero precision and recall even at very high thresholds, while **SVM and Logistic Regression** see these metrics drop to near zero?

**Hint:** Think about how each model produces probability estimates — are they smooth and continuous, or discrete/step-like?

### A2. Threshold Effects – Classifier-Centric View

Here, each panel shows a single classifier with **all metrics plotted** as the threshold changes.

**Purpose:**

* See how different metrics behave **for the same classifier**.
* Observe **trade-offs** and **sensitivity to threshold changes** within a classifier.
* Compare patterns between classifiers to understand **linear vs non-linear behavior**.

**Your task:**

* Examine the plots for each classifier and **compare across teams**.
* **Broad question:** How does each classifier respond to threshold changes across metrics, and what differences do you notice between classifiers?

> Focus on overall patterns, trade-offs, and differences rather than exact values.

In [ ]:
# Plot metrics vs threshold for each model
fig, axes = plt.subplots(2, 2, figsize=(12,10))
for ax, (name, df) in zip(axes.ravel(), results.items()):
    ax.plot(df['threshold'], df['accuracy'], label='accuracy')
    ax.plot(df['threshold'], df['precision'], label='precision')
    ax.plot(df['threshold'], df['recall'], label='recall')
    ax.plot(df['threshold'], df['f1'], label='f1')
    ax.set_title(f'{name.upper()} — metrics vs threshold')
    ax.set_xlabel('threshold')
    ax.set_xlim(0,1)
    ax.legend()
plt.tight_layout()
plt.show()

### Exploring the dataset properties and how it impacts the decision boundary.

Try a harder dataset and see how **decision boundaries** change.

In [ ]:
class_sep= 0.7    # less separation 1 is easy, o → more overlap
flip_y=0.05     #  noisy labels
weights=(0.95, 0.05)


X_train, X_test, y_train, y_test, X_train_scaled, X_test_scaled, fitted = make_experiment(class_sep=class_sep, flip_y=flip_y, weights=weights)


Look into the code below, what is the selection rationale?

In [ ]:
# Find borderline cases using logistic regression
probs_lr = fitted['logreg'].predict_proba(X_test)[:, 1]

# indices where model is uncertain (probability ~0.5)
borderline_idx = np.argsort(np.abs(probs_lr - 0.5))[:3]

X_sample = X_test_scaled[borderline_idx]
y_sample = y_test[borderline_idx]

print("Selected sample indices (borderline cases):", borderline_idx.tolist())


Now print the probabilities for these data samples, how does it look like?

In [ ]:
for name, model in fitted.items():
    if name in ('knn', 'svm'):
        probs = model.predict_proba(X_sample)[:, 1]
    else:
        probs = model.predict_proba(X_test[borderline_idx])[:, 1]

    print(f"\n{name.upper()} predictions (borderline samples):")
    for i, (true_label, prob) in enumerate(zip(y_sample, probs)):
        pred_default = int(prob >= 0.5)
        print(f"Sample {i}: True={true_label}, Prob={prob:.2f}, Pred@0.5={pred_default}")


Now compare how the threshold impact changes when dataset is hard.

---

🔑 Key Concept: ROC AUC and Average Precision (PR AUC)

So far we looked at metrics like accuracy, precision, recall, F1 at a fixed threshold (usually 0.5).
But what if we don’t want to commit to a single threshold yet?

That’s where threshold-independent metrics like ROC AUC and PR AUC come in.

1. ROC AUC (Area Under the ROC Curve):

ROC = Receiver Operating Characteristic curve.

Plots True Positive Rate (Recall) vs False Positive Rate (FPR) for all possible thresholds.

AUC (area under the curve): the probability that a randomly chosen positive is ranked higher than a randomly chosen negative.

Values:

0.5 = random guessing

1.0 = perfect ranking

👉 Good for balanced datasets, but can be overly optimistic when classes are very imbalanced.

2. PR AUC (Area under the Precision–Recall curve):

Plots Precision vs Recall across thresholds.

Average Precision (AP) is the area under this curve.

Much more informative than ROC AUC when the positive class is rare.

Values:

Close to the positive class prevalence = bad

Closer to 1.0 = good

👉 PR AUC focuses on how well the model finds the minority class without too many false alarms.




In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc

plt.figure(figsize=(12,5))

# ROC curve
plt.subplot(1,2,1)
for name, model in fitted.items():
    if name in ('knn','svm'):
        probs = model.predict_proba(X_test_scaled)[:,1]
    else:
        probs = model.predict_proba(X_test)[:,1]
    fpr, tpr, _ = roc_curve(y_test, probs)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{name.upper()} (AUC={roc_auc:.2f})')

plt.plot([0,1],[0,1],'--', color='gray')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()

# PR curve
plt.subplot(1,2,2)
for name, model in fitted.items():
    if name in ('knn','svm'):
        probs = model.predict_proba(X_test_scaled)[:,1]
    else:
        probs = model.predict_proba(X_test)[:,1]
    precision, recall, _ = precision_recall_curve(y_test, probs)
    pr_auc = auc(recall, precision)
    pr_auc=  average_precision_score(y_test, probs)
    plt.plot(recall, precision, lw=2, label=f'{name.upper()} (AP={pr_auc:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.tight_layout()
plt.show()

Note that metrics maybe inconsistent! You are lucky if all metrics tell you the same!

---

### Small-sample effects
Fit models with small training sets (n=50, 100, 300, 1000) and plot variability.

**Exercise C1:** How does metric variance change as sample size increases? Which metric is unstable?



In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

def small_sample_variability_decision_tree(sample_sizes=[50,100,300,1000], repeats=50, random_state=0):
    out = []
    for n in sample_sizes:
        sss = StratifiedShuffleSplit(n_splits=repeats, train_size=n, random_state=random_state)
        for i, (sample_idx, _) in enumerate(sss.split(X, y)):
            Xs, ys = X[sample_idx], y[sample_idx]
            Xtr, Xv, ytr, yv = train_test_split(
                Xs, ys, test_size=0.3, stratify=ys, random_state=i
            )

            clf = DecisionTreeClassifier(max_depth=3, random_state=random_state)
            clf.fit(Xtr, ytr)
            preds = clf.predict(X_test)

            out.append({
                'n': n,
                'repeat': i,
                'acc': accuracy_score(y_test, preds),
                'f1': f1_score(y_test, preds, zero_division=0),
                'recall': recall_score(y_test, preds, zero_division=0)
            })
    return pd.DataFrame(out)


# Run experiment
var_df = small_sample_variability_decision_tree(sample_sizes=[50,100,300,1000], repeats=50, random_state=42)

# Boxplots
plt.figure(figsize=(12,4))
for i, metric in enumerate(["acc", "f1", "recall"], 1):
    plt.subplot(1,3,i)
    sns.boxplot(x="n", y=metric, data=var_df)
    plt.title(f"{metric.upper()} across sample sizes")
plt.tight_layout()
plt.show()


Study the impact of model difficulty on sample size requirements.

---

## Exploration tasks for students
- Change imbalance ratio in `generate_data()`.
- Change `class_sep` to make the task harder/easier.




---

## 6. Reference: scikit-learn functions to explore
- **Data & Splitting:** `make_classification`, `train_test_split`  
- **Models:** `KNeighborsClassifier`, `DecisionTreeClassifier`, `SVC`, `LogisticRegression`  
- **Metrics:** `confusion_matrix`, `classification_report`, `precision_score`, `recall_score`,
  `f1_score`, `accuracy_score`, `roc_curve`, `roc_auc_score`, `precision_recall_curve`,
  `average_precision_score`, `calibration_curve`  
- **Utilities:** `StandardScaler`, `Pipeline`, `StratifiedKFold`, `cross_val_score`, `CalibratedClassifierCV`  

